In [25]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset/dakshina_dataset_v1.0/README.md
/kaggle/input/dataset/dakshina_dataset_v1.0/mr/lexicons/mr.translit.sampled.test.tsv
/kaggle/input/dataset/dakshina_dataset_v1.0/mr/lexicons/mr.translit.sampled.train.tsv
/kaggle/input/dataset/dakshina_dataset_v1.0/mr/lexicons/mr.translit.sampled.dev.tsv
/kaggle/input/dataset/dakshina_dataset_v1.0/mr/native_script_wikipedia/mr.wiki-filt.train.text.sorted.tsv/wiki-filt.train.text.sorted.tsv
/kaggle/input/dataset/dakshina_dataset_v1.0/mr/native_script_wikipedia/mr.wiki-full.nonblock.sections.list.txt/wiki-full.nonblock.sections.list.txt
/kaggle/input/dataset/dakshina_dataset_v1.0/mr/native_script_wikipedia/mr.wiki-filt.valid.text.shuf.txt/wiki-filt.valid.text.shuf.txt
/kaggle/input/dataset/dakshina_dataset_v1.0/mr/native_script_wikipedia/mr.wiki-full.urls.tsv/wiki-full.urls.tsv
/kaggle/input/dataset/dakshina_dataset_v1.0/mr/native_script_wikipedia/mr.wiki-full.omit_pages.txt/wiki-full.omit_pages.txt
/kaggle/input/dataset/dakshina_data

In [26]:
import os
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import wandb
import matplotlib.pyplot as plt
from tqdm import tqdm
from types import SimpleNamespace
import logging
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import time
import re

In [27]:
wandb.login(key="367b223a3f0954311018074733b3ed8dc409c2ff")
# Constants
wandb_project = "DL_A3_seq2seq"
wandb_entity= "da24m006-iit-madras"  

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [29]:
# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [30]:
data_dir = '/kaggle/input/dataset/dakshina_dataset_v1.0/hi/lexicons'
traindata = pd.read_csv(f'{data_dir}/hi.translit.sampled.train.tsv',sep='\t',names=['native', 'latin', 'count'],usecols=[0, 1])
valdata= pd.read_csv(f'{data_dir}/hi.translit.sampled.dev.tsv',sep='\t',names=['native', 'latin', 'count'],usecols=[0, 1])
testdata = pd.read_csv(f'{data_dir}/hi.translit.sampled.test.tsv',sep='\t',names=['native', 'latin', 'count'],usecols=[0, 1])

In [31]:
traindata

,native,latin
0,अं,an
1,अंकगणित,ankganit
2,अंकल,uncle
3,अंकुर,ankur
4,अंकुरण,ankuran
...,...,...
44199,ह्वेनसांग,hiuentsang
44200,ह्वेनसांग,hsuantsang
44201,ह्वेनसांग,hyensang
44202,ह्वेनसांग,xuanzang


In [32]:
print("Total Dataset: ")
print("Total train data:", len(traindata))
print("Total validation data:", len(valdata))
print("Totat test data:", len(testdata))

Total Dataset: 
Total train data: 44204
Total validation data: 4358
Totat test data: 4502


In [33]:
Hindi_vocab = set()
max_hin_len = 0

# Process training data
for x in range(len(traindata)):
    native_word = traindata.iloc[x]['native']
    if pd.isna(native_word):
        continue

    max_hin_len = max(max_hin_len, len(native_word))
    for char in native_word:
        Hindi_vocab.add(char)

# Process test data
for x in range(len(testdata)):
    native_word = testdata.iloc[x]['native']
    if pd.isna(native_word):
        continue

    for char in native_word:
        if char not in Hindi_vocab:
            print(char)
            Hindi_vocab.add(char)


Hindi_vocab = sorted(Hindi_vocab)

print("Max Hindi word length:", max_hin_len)
print("Total unique Hindi characters:", len(Hindi_vocab))
print(Hindi_vocab)

Max Hindi word length: 19
Total unique Hindi characters: 63
['ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऐ', 'ऑ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह', '़', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॅ', 'े', 'ै', 'ॉ', 'ो', 'ौ', '्', 'ॐ']


In [34]:
max_eng_len = 0
English_vocab = set() 

for x in range(len(traindata)):
    latin_word = traindata.iloc[x]['latin']
    
    if pd.isna(latin_word):
        continue  # skip if value is missing

    word_len = len(latin_word)
    max_eng_len = max(max_eng_len, word_len)

    for char in latin_word:
        English_vocab.add(char)

# Convert set to sorted list
English_vocab = sorted(English_vocab)
print("Max english word length:", max_eng_len)
print("Total unique english characters:", len(English_vocab))
print(English_vocab)


Max english word length: 20
Total unique english characters: 26
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


##### Data Loader (Data Preprocessing) 

In [35]:
class TransliterationDataset(Dataset):
    def __init__(self, file_path):
        self.source_texts = []
        self.target_texts = []
        
        # Check if file exists
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Data file not found: {file_path}")
            
        with open(file_path, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                parts = line.strip().split('\t')
                
                # Handle different file formats
                if len(parts) == 3:  # Format: Hindi\tRomanized\tFrequency
                    target, source, _ = parts
                    self.source_texts.append(source)
                    self.target_texts.append(target)
                elif len(parts) == 2:  # Format: Hindi\tRomanized
                    target, source = parts
                    self.source_texts.append(source)
                    self.target_texts.append(target)
                else:
                    print(f"Warning: Line {line_num} has unexpected format (expected 2 or 3 columns): {line.strip()}")
        
        # If no data was loaded, raise an error
        if len(self.source_texts) == 0:
            raise ValueError(f"No data was loaded from {file_path}. File may be empty or incorrectly formatted.")
                    
        self.source_char_set = set()
        self.target_char_set = set()
        
        for source in self.source_texts:
            self.source_char_set.update(source)
        
        for target in self.target_texts:
            self.target_char_set.update(target)
        
        self.source_char_to_idx = {char: idx + 1 for idx, char in enumerate(sorted(self.source_char_set))}
        self.source_idx_to_char = {idx + 1: char for idx, char in enumerate(sorted(self.source_char_set))}
        self.source_char_to_idx['<PAD>'] = 0
        self.source_idx_to_char[0] = '<PAD>'
        self.source_char_to_idx['< SOS >'] = len(self.source_char_to_idx)
        self.source_idx_to_char[len(self.source_idx_to_char)] = '< SOS >'
        self.source_char_to_idx['<EOS>'] = len(self.source_char_to_idx)
        self.source_idx_to_char[len(self.source_idx_to_char)] = '<EOS>'
        
        self.target_char_to_idx = {char: idx + 1 for idx, char in enumerate(sorted(self.target_char_set))}
        self.target_idx_to_char = {idx + 1: char for idx, char in enumerate(sorted(self.target_char_set))}
        self.target_char_to_idx['<PAD>'] = 0
        self.target_idx_to_char[0] = '<PAD>'
        self.target_char_to_idx['< SOS >'] = len(self.target_char_to_idx)
        self.target_idx_to_char[len(self.target_idx_to_char)] = '< SOS >'
        self.target_char_to_idx['<EOS>'] = len(self.target_char_to_idx)
        self.target_idx_to_char[len(self.target_idx_to_char)] = '<EOS>'
        
    def __len__(self):
        return len(self.source_texts)
    
    def __getitem__(self, idx):
        source = self.source_texts[idx]
        target = self.target_texts[idx]
        
        source_indices = [self.source_char_to_idx['< SOS >']] + [self.source_char_to_idx[char] for char in source] + [self.source_char_to_idx['<EOS>']]
        target_indices = [self.target_char_to_idx['< SOS >']] + [self.target_char_to_idx[char] for char in target] + [self.target_char_to_idx['<EOS>']]
        
        return {
            'source': source,
            'target': target,
            'source_indices': source_indices,
            'target_indices': target_indices,
            'source_length': len(source_indices),
            'target_length': len(target_indices)
        }
    
    @property
    def source_vocab_size(self):
        return len(self.source_char_to_idx)
    
    @property
    def target_vocab_size(self):
        return len(self.target_char_to_idx)

def collate_fn(batch):
    max_source_len = max([len(item['source_indices']) for item in batch])
    max_target_len = max([len(item['target_indices']) for item in batch])
    
    source_batch = []
    target_batch = []
    source_lengths = []
    target_lengths = []
    
    source_texts = []
    target_texts = []
    
    for item in batch:
        source_padded = item['source_indices'] + [0] * (max_source_len - len(item['source_indices']))
        target_padded = item['target_indices'] + [0] * (max_target_len - len(item['target_indices']))
        
        source_batch.append(source_padded)
        target_batch.append(target_padded)
        source_lengths.append(item['source_length'])
        target_lengths.append(item['target_length'])
        
        source_texts.append(item['source'])
        target_texts.append(item['target'])
    
    return {
        'source': torch.LongTensor(source_batch),
        'target': torch.LongTensor(target_batch),
        'source_lengths': torch.LongTensor(source_lengths),
        'target_lengths': torch.LongTensor(target_lengths),
        'source_texts': source_texts,
        'target_texts': target_texts
    }

def get_dataloader(file_path, batch_size, shuffle=True):
    try:
        dataset = TransliterationDataset(file_path)
        dataloader = DataLoader(
            dataset,
            batch_size=batch_size,
            shuffle=shuffle,
            collate_fn=collate_fn
        )
        return dataloader, dataset
    except FileNotFoundError as e:
        print(f"Error: {str(e)}")
        print(f"Please make sure the Dakshina dataset is downloaded and located correctly.")
        print(f"Expected file path: {file_path}")
        print(f"Current working directory: {os.getcwd()}")
        raise
    except ValueError as e:
        print(f"Error: {str(e)}")
        raise

##### Seq2Seq Model Vanila

In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

class Encoder(nn.Module):
    def __init__(self, input_size, embed_size, hidden_size, num_layers, dropout, cell_type="GRU"):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_size, embed_size)
        self.dropout = nn.Dropout(dropout)
        
        if cell_type == "GRU":
            self.rnn = nn.GRU(
                embed_size,
                hidden_size,
                num_layers,
                dropout=dropout if num_layers > 1 else 0,
                batch_first=True
            )
        elif cell_type == "LSTM":
            self.rnn = nn.LSTM(
                embed_size,
                hidden_size,
                num_layers,
                dropout=dropout if num_layers > 1 else 0,
                batch_first=True
            )
        else:  # Default to RNN
            self.rnn = nn.RNN(
                embed_size,
                hidden_size,
                num_layers,
                dropout=dropout if num_layers > 1 else 0,
                batch_first=True
            )
        
        self.cell_type = cell_type
    
    def forward(self, x, lengths):
        # x shape: (batch_size, seq_length)
        embedded = self.dropout(self.embedding(x))
        
        # Pack padded batch of sequences for RNN
        packed = nn.utils.rnn.pack_padded_sequence(
            embedded, lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        
        # Forward propagate RNN
        outputs, hidden = self.rnn(packed)
        
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs, batch_first=True)
        
        return outputs, hidden


class Decoder(nn.Module):
    def __init__(self, output_size, embed_size, hidden_size, num_layers, dropout, cell_type="GRU"):
        super(Decoder, self).__init__()
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.cell_type = cell_type
        
        self.embedding = nn.Embedding(output_size, embed_size)
        self.dropout = nn.Dropout(dropout)
        
        if cell_type == "GRU":
            self.rnn = nn.GRU(
                embed_size,
                hidden_size,
                num_layers,
                dropout=dropout if num_layers > 1 else 0,
                batch_first=True
            )
        elif cell_type == "LSTM":
            self.rnn = nn.LSTM(
                embed_size,
                hidden_size,
                num_layers,
                dropout=dropout if num_layers > 1 else 0,
                batch_first=True
            )
        else:  # Default to RNN
            self.rnn = nn.RNN(
                embed_size,
                hidden_size,
                num_layers,
                dropout=dropout if num_layers > 1 else 0,
                batch_first=True
            )
        
        self.fc_out = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, hidden):
        # x shape: (batch_size, 1)
        embedded = self.dropout(self.embedding(x))
        
        # Forward propagate through RNN
        output, hidden = self.rnn(embedded, hidden)
        
        # Compute output
        prediction = self.fc_out(output.squeeze(1))
        
        return prediction, hidden


class Seq2Seq(nn.Module):
    def __init__(self, 
                 input_size, 
                 output_size, 
                 embed_size, 
                 hidden_size, 
                 num_encoder_layers, 
                 num_decoder_layers, 
                 dropout, 
                 cell_type="GRU"):
        super(Seq2Seq, self).__init__()
        
        self.encoder = Encoder(
            input_size, 
            embed_size, 
            hidden_size, 
            num_encoder_layers, 
            dropout, 
            cell_type
        )
        
        self.decoder = Decoder(
            output_size, 
            embed_size, 
            hidden_size, 
            num_decoder_layers, 
            dropout, 
            cell_type
        )
        
        self.cell_type = cell_type
        self.num_encoder_layers = num_encoder_layers
        self.num_decoder_layers = num_decoder_layers
        self.hidden_size = hidden_size
    
    def forward(self, source, target, source_lengths, teacher_forcing_ratio=0.5):
        batch_size = source.shape[0]
        target_len = target.shape[1]
        target_vocab_size = self.decoder.output_size
        
        # Tensor to store decoder outputs
        outputs = torch.zeros(batch_size, target_len, target_vocab_size).to(source.device)
        
        # Encode source sequences
        encoder_outputs, hidden = self.encoder(source, source_lengths)
        
        # Process hidden state if encoder and decoder layers differ
        hidden = self._process_hidden_for_decoder(hidden, batch_size)
        
        # First input to the decoder is the < SOS > token
        decoder_input = target[:, 0].unsqueeze(1)
        
        # Decode one character at a time
        for t in range(1, target_len):
            output, hidden = self.decoder(decoder_input, hidden)
            outputs[:, t, :] = output
            
            # Teacher forcing: decide whether to use the predicted or actual target as next input
            teacher_force = random.random() < teacher_forcing_ratio
            
            # Get the highest predicted token
            top1 = output.argmax(1)
            
            # If teacher forcing, use actual next token as input; otherwise use predicted token
            decoder_input = target[:, t].unsqueeze(1) if teacher_force else top1.unsqueeze(1)
        
        return outputs
    
    def _process_hidden_for_decoder(self, hidden, batch_size):
        """Process the encoder hidden state for the decoder"""
        if self.num_encoder_layers == self.num_decoder_layers:
            return hidden
        
        # For GRU and RNN
        if self.cell_type != "LSTM":
            if self.num_encoder_layers < self.num_decoder_layers:
                # Duplicate last layer to match decoder layers
                additional_layers = self.num_decoder_layers - self.num_encoder_layers
                last_layer = hidden[-1:].expand(additional_layers, batch_size, self.hidden_size)
                return torch.cat([hidden, last_layer], dim=0)
            else:
                # Use only the last n layers
                return hidden[-self.num_decoder_layers:]
        else:
            # For LSTM (hidden is a tuple of (h_0, c_0))
            h, c = hidden
            if self.num_encoder_layers < self.num_decoder_layers:
                # Duplicate last layer to match decoder layers
                additional_layers = self.num_decoder_layers - self.num_encoder_layers
                last_h_layer = h[-1:].expand(additional_layers, batch_size, self.hidden_size)
                last_c_layer = c[-1:].expand(additional_layers, batch_size, self.hidden_size)
                new_h = torch.cat([h, last_h_layer], dim=0)
                new_c = torch.cat([c, last_c_layer], dim=0)
                return (new_h, new_c)
            else:
                # Use only the last n layers
                return (h[-self.num_decoder_layers:], c[-self.num_decoder_layers:])
    
    def predict(self, source, source_len, target_vocab_size, sos_idx, eos_idx, max_len=100):
        batch_size = source.shape[0]
        
        # Encode source sequences
        encoder_outputs, hidden = self.encoder(source, source_len)
        
        # Process hidden state if encoder and decoder layers differ
        hidden = self._process_hidden_for_decoder(hidden, batch_size)
        
        # First input to the decoder is the < SOS > token
        decoder_input = torch.tensor([[sos_idx]] * batch_size).to(source.device)
        
        # Lists to store predicted outputs
        predictions = []
        
        # Flag to indicate if decoding is complete
        done = [False] * batch_size
        
        # Decode one character at a time
        for _ in range(max_len):
            output, hidden = self.decoder(decoder_input, hidden)
            
            # Get the highest predicted token
            top1 = output.argmax(1)
            
            # Store predicted token
            predictions.append(top1.unsqueeze(1))
            
            # Check if all sequences have reached EOS
            for i in range(batch_size):
                if top1[i].item() == eos_idx:
                    done[i] = True
            
            if all(done):
                break
            
            # Use predicted token as next input
            decoder_input = top1.unsqueeze(1)
        
        # Concatenate predictions along the sequence dimension
        return torch.cat(predictions, dim=1)

##### utilities function used in training 

In [37]:
import torch
import os
import numpy as np

def calculate_accuracy(predictions, targets, target_lengths, ignore_index=0):
    """
    Calculate the accuracy of the predictions against the targets.
    A prediction is considered correct only if all characters in the sequence match.
    """
    batch_size = predictions.size(0)
    correct = 0
    
    for i in range(batch_size):
        pred_seq = predictions[i].cpu().numpy()
        target_seq = targets[i, 1:target_lengths[i]-1].cpu().numpy()  # Exclude SOS and EOS tokens
        
        # Check if prediction exactly matches target
        if len(pred_seq) == len(target_seq) and np.array_equal(pred_seq, target_seq):
            correct += 1
    
    return correct / batch_size

def save_checkpoint(model, optimizer, epoch, accuracy, filename):
    """Save model checkpoint"""
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'accuracy': accuracy
    }, filename)

def load_checkpoint(model, optimizer, filename):
    """Load model checkpoint"""
    checkpoint = torch.load(filename)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    accuracy = checkpoint.get('accuracy', 0.0)
    return model, optimizer, epoch, accuracy

def indices_to_string(indices, idx_to_char, eos_idx=None):
    """Convert a sequence of indices to a string"""
    if eos_idx is not None:
        # Find the index of the first EOS token
        try:
            eos_pos = indices.index(eos_idx)
            indices = indices[:eos_pos]
        except ValueError:
            pass  # No EOS token found
    
    return ''.join([idx_to_char[idx] for idx in indices if idx in idx_to_char and idx_to_char[idx] not in ['<PAD>', '< SOS >', '<EOS>']])

def create_directory(directory):
    """Create directory if it doesn't exist"""
    if not os.path.exists(directory):
        os.makedirs(directory)

##### config for training on a fixed parameter(model,training)

In [13]:
import os

class Config:
    # Data parameters
    language = os.environ.get("TRANSLITERATION_LANGUAGE", "hi")  # Get language from env var or default to "hi"
    #base_dir = os.path.dirname(os.path.abspath(__file__))  # Get the directory of the current file
    #data_dir = os.path.join(base_dir, "dakshina_dataset_v1.0", f"{language}", "lexicons") 
    input_dir = "/kaggle/input/dataset"  # Read-only input path
    base_dir = "/kaggle/working"         # Writable path for outputs
    
    data_dir = os.path.join(input_dir, "dakshina_dataset_v1.0", language, "lexicons")
    # Check if data directory exists, if not, try to find it elsewhere
    if not os.path.exists(data_dir):
        # Try to find the dakshina dataset in parent directories
        parent_dir = os.path.dirname(base_dir)
        alt_data_dir = os.path.join(parent_dir, "dakshina_dataset_v1.0", f"{language}", "lexicons")
        if os.path.exists(alt_data_dir):
            data_dir = alt_data_dir
    
    train_file = os.path.join(data_dir, f"{language}.translit.sampled.train.tsv")
    val_file = os.path.join(data_dir, f"{language}.translit.sampled.dev.tsv")
    test_file = os.path.join(data_dir, f"{language}.translit.sampled.test.tsv")
    
    # Model parameters (default values)
    embed_size = 64
    hidden_size = 128
    num_encoder_layers = 1
    num_decoder_layers = 1
    dropout = 0.2
    cell_type = "GRU"  # Options: RNN, LSTM, GRU
    
    # Training parameters
    batch_size = 64
    epochs = 20
    learning_rate = 0.001
    teacher_forcing_ratio = 0.5
    
    # Decoding parameters
    beam_size = 1  # 1 for greedy decoding
    
    # W&B parameters
    wandb_project = "DL_A3_seq2seq"
    wandb_entity = "da24m006-iit-madras"  
    
    # Save directories
    model_dir = os.path.join(base_dir, "saved_models", language)
    # Create directory for saving model if it doesn't exist
    if not os.path.exists(model_dir):
        os.makedirs(model_dir, exist_ok=True)
    
    prediction_dir = os.path.join(base_dir, "predictions", language)
    # Create directory for saving prediction if it doesn't exist
    if not os.path.exists(prediction_dir):
        os.makedirs(prediction_dir, exist_ok=True)

    
    def __init__(self):
        # Verify paths and print info
        print(f"Base directory: {self.base_dir}")
        print(f"Language: {self.language}")
        print(f"Data directory: {self.data_dir}")
        print(f"Train file exists: {os.path.exists(self.train_file)}")
        print(f"Val file exists: {os.path.exists(self.val_file)}")
        print(f"Test file exists: {os.path.exists(self.test_file)}")
        
        # Print sample data from files if they exist
        self._print_sample_data(self.train_file, "train")
        self._print_sample_data(self.val_file, "validation")
        self._print_sample_data(self.test_file, "test")
    
    def _print_sample_data(self, file_path, name):
        """Print a sample of data from the file for debugging"""
        if os.path.exists(file_path):
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    lines = f.readlines()[:3]  # Get first 3 lines
                    if lines:
                        print(f"\nSample {name} data:")
                        for line in lines:
                            print(f"  {line.strip()}")
                        print(f"  ... (total lines: {sum(1 for _ in open(file_path, 'r', encoding='utf-8'))})")
                    else:
                        print(f"\nWarning: {name} file exists but is empty: {file_path}")
            except Exception as e:
                print(f"\nError reading {name} file: {str(e)}")

##### Training with a fixed config

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import wandb
import os
import time
from tqdm import tqdm

# from config import Config
# from dataloader import get_dataloader
# from seq2seq import Seq2Seq
# from utils import save_checkpoint, create_directory, indices_to_string

def train(config, run_name=None, run_already_initialized=False):
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Initialize W&B only if it's not already initialized
    if not run_already_initialized:
        if run_name:
            wandb.init(project=config.wandb_project, entity=config.wandb_entity, name=run_name, config=vars(config))
        else:
            # Include language in the run name if not provided
            default_run_name = f"{config.cell_type}_lang_{config.language}_e{config.embed_size}_h{config.hidden_size}"
            wandb.init(project=config.wandb_project, entity=config.wandb_entity, name=default_run_name, config=vars(config))
    
        # Explicitly log the language being used
        wandb.config.update({"language": config.language})
    
    print(f"\nTraining transliteration model for language: {config.language}")
    print(f"Using dataset from: {config.data_dir}\n")
    
    try:
        # Load data
        train_loader, train_dataset = get_dataloader(
            config.train_file, 
            config.batch_size, 
            shuffle=True
        )
        
        val_loader, val_dataset = get_dataloader(
            config.val_file, 
            config.batch_size, 
            shuffle=False
        )
        
        # Create model
        model = Seq2Seq(
            input_size=train_dataset.source_vocab_size,
            output_size=train_dataset.target_vocab_size,
            embed_size=config.embed_size,
            hidden_size=config.hidden_size,
            num_encoder_layers=config.num_encoder_layers,
            num_decoder_layers=config.num_decoder_layers,
            dropout=config.dropout,
            cell_type=config.cell_type
        )
        
        model = model.to(device)
        
        # Define loss function and optimizer
        criterion = nn.CrossEntropyLoss(ignore_index=0)  # Ignore padding index
        optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
        
        # Create directory for saving models
        create_directory(config.model_dir)
        
        best_accuracy = 0.0
        
        # Training loop
        for epoch in range(config.epochs):
            model.train()
            total_loss = 0
            
            pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{config.epochs}')
            
            for batch_idx, batch in enumerate(pbar):
                # Move tensors to device
                source = batch['source'].to(device)
                target = batch['target'].to(device)
                source_lengths = batch['source_lengths']
                
                # Zero the gradients
                optimizer.zero_grad()
                
                # Forward pass
                output = model(source, target, source_lengths, config.teacher_forcing_ratio)
                
                # Calculate loss
                # Reshape output and target for loss calculation
                output_dim = output.shape[-1]
                output = output[:, 1:].reshape(-1, output_dim)  # Remove SOS token
                target = target[:, 1:].reshape(-1)  # Remove SOS token
                
                # Calculate loss
                loss = criterion(output, target)
                
                # Backward pass and optimize
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Clip gradients
                optimizer.step()
                
                total_loss += loss.item()
                pbar.set_postfix({'loss': loss.item()})
            
            # Calculate average loss
            avg_loss = total_loss / len(train_loader)
            
            # Evaluate on validation set
            model.eval()
            val_loss, val_accuracy = evaluate(model, val_loader, criterion, device, train_dataset)
            
            # Log metrics to W&B
            wandb.log({
                'epoch': epoch + 1,
                'train_loss': avg_loss,
                'val_loss': val_loss,
                'val_accuracy': val_accuracy,
                'language': config.language  # Log language with metrics
            })
            
            print(f'Epoch: {epoch+1}, Train Loss: {avg_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')
            
            # Save model if it's the best so far
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
                if not os.path.exists(os.path.join(config.model_dir)):
                    os.makedirs(os.path.join(config.model_dir))

                # Save locally first with language code in filename
                model_path = os.path.join(config.model_dir, f'best_model_{config.language}_{wandb.run.id}.pt')
                save_checkpoint(
                    model, 
                    optimizer, 
                    epoch, 
                    val_accuracy, 
                    model_path
                )
                
                # Save model state dictionary to W&B
                try:
                    # Save model for W&B
                    wandb_model_path = os.path.join(wandb.run.dir, "best_model.pt")
                    torch.save(model.state_dict(), wandb_model_path)
                    
                    # Use W&B Artifact API instead of direct file save
                    artifact = wandb.Artifact(f"model-{config.language}-{wandb.run.id}", type="model")
                    artifact.add_file(wandb_model_path)
                    wandb.log_artifact(artifact)
                    
                    print(f"Model successfully saved to W&B as artifact")
                except Exception as e:
                    print(f"Failed to save model to W&B: {str(e)}")
                    print(f"Model was saved locally to {model_path}")
            
            # Save checkpoint
            if (epoch + 1) % 5 == 0:
                checkpoint_path = os.path.join(config.model_dir, f'checkpoint_{config.language}_{epoch+1}_{wandb.run.id}.pt')
                save_checkpoint(
                    model, 
                    optimizer, 
                    epoch, 
                    val_accuracy, 
                    checkpoint_path
                )

        # Only finish wandb if we started it in this function
        if not run_already_initialized:
            wandb.finish()
        
        return model, best_accuracy
        
    except Exception as e:
        print(f"Error during training: {str(e)}")
        # Only finish wandb if we started it in this function
        if not run_already_initialized:
            wandb.finish()
        # Re-raise the exception for proper error handling
        raise

def evaluate(model, dataloader, criterion, device, dataset):
    model.eval()
    total_loss = 0
    total_correct = 0
    total_examples = 0
    
    with torch.no_grad():
        for batch in dataloader:
            # Move tensors to device
            source = batch['source'].to(device)
            target = batch['target'].to(device)
            source_lengths = batch['source_lengths']
            target_lengths = batch['target_lengths']
            
            # Forward pass
            output = model(source, target, source_lengths, teacher_forcing_ratio=0)
            
            # Calculate loss
            output_dim = output.shape[-1]
            output_for_loss = output[:, 1:].reshape(-1, output_dim)  # Remove SOS token
            target_for_loss = target[:, 1:].reshape(-1)  # Remove SOS token
            
            loss = criterion(output_for_loss, target_for_loss)
            total_loss += loss.item()
            
            # Generate predictions
            predictions = model.predict(
                source,
                source_lengths,
                dataset.target_vocab_size,
                dataset.target_char_to_idx['< SOS >'],
                dataset.target_char_to_idx['<EOS>']
            )
            
            # Calculate accuracy (exact match)
            for i in range(len(predictions)):
                pred_seq = [idx.item() for idx in predictions[i]]
                pred_str = indices_to_string(pred_seq, dataset.target_idx_to_char, dataset.target_char_to_idx['<EOS>'])
                target_str = batch['target_texts'][i]
                
                if pred_str == target_str:
                    total_correct += 1
            
            total_examples += len(predictions)
    
    avg_loss = total_loss / len(dataloader)
    accuracy = total_correct / total_examples
    
    return avg_loss, accuracy

# if __name__ == "__main__":
#     config = Config()
#     train(config)

Sweep 

In [15]:
##### import wandb
import os
import argparse
# from config import Config
# from train import train

sweep_configuration = {
    'method': 'bayes',  # or 'grid' or 'random'
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'embed_size': {
            'values': [16, 32, 64, 128, 256]
        },
        'hidden_size': {
            'values': [32, 64, 128, 256, 512]
        },
        'num_encoder_layers': {
            'values': [1, 2, 3]
        },
        'num_decoder_layers': {
            'values': [1, 2, 3]
        },
        'cell_type': {
            'values': ['RNN', 'GRU', 'LSTM']
        },
        'dropout': {
            'values': [0.1, 0.2, 0.3, 0.5]
        },
        'learning_rate': {
            'values': [0.0001, 0.0005, 0.001]
        },
        'batch_size': {
            'values': [32, 64, 128]
        },
        'teacher_forcing_ratio': {
            'values': [0.3, 0.5, 0.7]
        }
    }
}
def sweep_train(config_defaults=None):
    config = Config()  # initialize config early to construct run_name

    # Construct run name before initializing wandb
    run_name = f"{config.cell_type}_{config.language}_drop{config.dropout}_es{config.embed_size}_hs{config.hidden_size}_en{config.num_encoder_layers}_de{config.num_decoder_layers}_lr{config.learning_rate}"
    
    with wandb.init(config=config_defaults, name=run_name) as run:
        wandb_config = wandb.config
        for key, value in wandb_config.items():
            if hasattr(config, key):
                setattr(config, key, value)
        
        # Update run_name again after sweep parameters are applied
        run_name = f"{config.cell_type}_{config.language}_drop{config.dropout}_es{config.embed_size}_hs{config.hidden_size}_en{config.num_encoder_layers}_de{config.num_decoder_layers}_lr{config.learning_rate}"
        wandb.run.name = run_name
        wandb.run.save()

        # Train the model
        model, accuracy = train(config, run_already_initialized=True)

        # Log final result
        run.summary['best_accuracy'] = accuracy

# def sweep_train(config_defaults=None):
#     # Initialize a new wandb run
#     with wandb.init(config=config_defaults) as run:
#         # Copy hyperparameters from wandb config
#         config = Config()


#         # Create model name based on parameters
#         run_name = f"{config.cell_type}_lang_{config.language}_drop{config.dropout}_es{config.embed_size}_hs{config.hidden_size}_en{config.num_encoder_layers}_de{config.num_decoder_layers}_lr{config.learning_rate}"
                
#         # Update config with parameters from sweep
#         wandb_config = wandb.config
#         for key, value in wandb_config.items():
#             if hasattr(config, key):
#                 setattr(config, key, value)
        
        
#         # Train the model
#         # Note: We don't pass run_name here as wandb.init() is already handled in the train function
#         # and we're using the current run within this context
#         model, accuracy = train(config, run_already_initialized=True)
        
#         # Log the final result to the current run
#         run.summary['best_accuracy'] = accuracy

# def main():
#     parser = argparse.ArgumentParser(description='Run W&B sweep for transliteration model')
#     parser.add_argument('--sweep_id', type=str, help='The sweep ID to use (if already created)')
#     parser.add_argument('--count', type=int, default=40, help='Number of sweep runs')
#     parser.add_argument('--language', type=str, default='hi', 
#                         choices=['bn', 'gu', 'hi', 'kn', 'ml', 'mr', 'pa', 'sd', 'si', 'ta', 'te', 'ur'],
#                         help='Language code to use (default: hi)')
#     args = parser.parse_args()
    
#     # Set language in environment variable so config.py can access it
#     os.environ["TRANSLITERATION_LANGUAGE"] = args.language
    
#     # Set up W&B project
#     config = Config()
    
#     if args.sweep_id:
#         sweep_id = args.sweep_id
#     else:
#         sweep_id = wandb.sweep(sweep=sweep_configuration, project=config.wandb_project, entity=config.wandb_entity)
    
#     # Start sweep agent
#     wandb.agent(sweep_id, function=sweep_train, count=args.count)

# if __name__ == "__main__":
#     main()
# os.environ["TRANSLITERATION_LANGUAGE"] = "hi"
# config = Config()

# sweep_id = wandb.sweep(sweep=sweep_configuration, project=config.wandb_project, entity=config.wandb_entity)
# wandb.agent(sweep_id, function=sweep_train, count=50)

Base directory: /kaggle/working
Language: hi
Data directory: /kaggle/input/dataset/dakshina_dataset_v1.0/hi/lexicons
Train file exists: True
Val file exists: True
Test file exists: True

Sample train data:
  अं	an	3
  अंकगणित	ankganit	3
  अंकल	uncle	4
  ... (total lines: 44204)

Sample validation data:
  अंकन	ankan	3
  अंगकोर	angkor	3
  अंगिरा	angira	3
  ... (total lines: 4358)

Sample test data:
  अंक	ank	5
  अंक	anka	1
  अंकित	ankit	3
  ... (total lines: 4502)
Create sweep with ID: 2nykpam1
Sweep URL: https://wandb.ai/da24m006-iit-madras/DL_A3_seq2seq/sweeps/2nykpam1


wandb: Agent Starting Run: 21hw6j60 with config:
wandb: 	batch_size: 128
wandb: 	cell_type: RNN
wandb: 	dropout: 0.1
wandb: 	embed_size: 256
wandb: 	hidden_size: 256
wandb: 	learning_rate: 0.001
wandb: 	num_decoder_layers: 3
wandb: 	num_encoder_layers: 1
wandb: 	teacher_forcing_ratio: 0.5


Base directory: /kaggle/working
Language: hi
Data directory: /kaggle/input/dataset/dakshina_dataset_v1.0/hi/lexicons
Train file exists: True
Val file exists: True
Test file exists: True

Sample train data:
  अं	an	3
  अंकगणित	ankganit	3
  अंकल	uncle	4
  ... (total lines: 44204)

Sample validation data:
  अंकन	ankan	3
  अंगकोर	angkor	3
  अंगिरा	angira	3
  ... (total lines: 4358)

Sample test data:
  अंक	ank	5
  अंक	anka	1
  अंकित	ankit	3
  ... (total lines: 4502)


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.



Training transliteration model for language: hi
Using dataset from: /kaggle/input/dataset/dakshina_dataset_v1.0/hi/lexicons



...

Epoch 19/20: 100%|██████████| 346/346 [00:10<00:00, 33.68it/s, loss=0.873]


Epoch: 19, Train Loss: 1.0084, Val Loss: 7.8390, Val Accuracy: 0.2198
Model successfully saved to W&B as artifact


Epoch 20/20: 100%|██████████| 346/346 [00:10<00:00, 32.12it/s, loss=0.974]


Epoch: 20, Train Loss: 0.9816, Val Loss: 7.8761, Val Accuracy: 0.2299
Model successfully saved to W&B as artifact


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train_loss,█▇▆▅▄▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁
val_accuracy,▁▁▁▁▂▂▃▃▄▄▅▅▆▆▆▇▇███
val_loss,▁▁▂▂▂▃▃▄▄▅▅▆▆▇▇▇▇███
best_accuracy,0.22992
epoch,20
language,hi
train_loss,0.98164
val_accuracy,0.22992
val_loss,7.87605


Q4: Test uisng  Best Model  

In [38]:
import os

class Config:
    # Data parameters
    language = os.environ.get("TRANSLITERATION_LANGUAGE", "hi")  # Get language from env var or default to "hi"
    #base_dir = os.path.dirname(os.path.abspath(__file__))  # Get the directory of the current file
    #data_dir = os.path.join(base_dir, "dakshina_dataset_v1.0", f"{language}", "lexicons") 
    input_dir = "/kaggle/input/dataset"  # Read-only input path
    base_dir = "/kaggle/working"         # Writable path for outputs
    
    data_dir = os.path.join(input_dir, "dakshina_dataset_v1.0", language, "lexicons")
    # Check if data directory exists, if not, try to find it elsewhere
    if not os.path.exists(data_dir):
        # Try to find the dakshina dataset in parent directories
        parent_dir = os.path.dirname(base_dir)
        alt_data_dir = os.path.join(parent_dir, "dakshina_dataset_v1.0", f"{language}", "lexicons")
        if os.path.exists(alt_data_dir):
            data_dir = alt_data_dir
    
    train_file = os.path.join(data_dir, f"{language}.translit.sampled.train.tsv")
    val_file = os.path.join(data_dir, f"{language}.translit.sampled.dev.tsv")
    test_file = os.path.join(data_dir, f"{language}.translit.sampled.test.tsv")
    
    # Model parameters 
    #Best model parameter
    embed_size = 32
    hidden_size = 512
    num_encoder_layers = 3
    num_decoder_layers = 3
    dropout = 0.5
    cell_type = "LSTM"  # Options: RNN, LSTM, GRU
    
    # Training parameters
    batch_size = 64
    epochs = 30
    learning_rate = 0.001
    teacher_forcing_ratio = 0.7
    
    # Decoding parameters
    beam_size = 1  # 1 for greedy decoding
    
    # W&B parameters
    wandb_project = "DL_A3_seq2seq"
    wandb_entity = "da24m006-iit-madras"  
    
    # Save directories
    model_dir = os.path.join(base_dir, "saved_models", language)
    # Create directory for saving model if it doesn't exist
    if not os.path.exists(model_dir):
        os.makedirs(model_dir, exist_ok=True)
    
    prediction_dir = os.path.join(base_dir, "predictions", language)
    # Create directory for saving prediction if it doesn't exist
    if not os.path.exists(prediction_dir):
        os.makedirs(prediction_dir, exist_ok=True)

    
    def __init__(self):
        # Verify paths and print info
        print(f"Base directory: {self.base_dir}")
        print(f"Language: {self.language}")
        print(f"Data directory: {self.data_dir}")
        print(f"Train file exists: {os.path.exists(self.train_file)}")
        print(f"Val file exists: {os.path.exists(self.val_file)}")
        print(f"Test file exists: {os.path.exists(self.test_file)}")
        
        # Print sample data from files if they exist
        self._print_sample_data(self.train_file, "train")
        self._print_sample_data(self.val_file, "validation")
        self._print_sample_data(self.test_file, "test")
    
    def _print_sample_data(self, file_path, name):
        """Print a sample of data from the file for debugging"""
        if os.path.exists(file_path):
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    lines = f.readlines()[:3]  # Get first 3 lines
                    if lines:
                        print(f"\nSample {name} data:")
                        for line in lines:
                            print(f"  {line.strip()}")
                        print(f"  ... (total lines: {sum(1 for _ in open(file_path, 'r', encoding='utf-8'))})")
                    else:
                        print(f"\nWarning: {name} file exists but is empty: {file_path}")
            except Exception as e:
                print(f"\nError reading {name} file: {str(e)}")

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import wandb
import os
import time
from tqdm import tqdm

# from config import Config
# from dataloader import get_dataloader
# from seq2seq import Seq2Seq
# from utils import save_checkpoint, create_directory, indices_to_string

def train(config, run_name=None, run_already_initialized=False):
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Initialize W&B only if it's not already initialized
    if not run_already_initialized:
        if run_name:
            wandb.init(project=config.wandb_project, entity=config.wandb_entity, name=run_name, config=vars(config))
        else:
            # Include language in the run name if not provided
            default_run_name = f"{config.cell_type}_lang_{config.language}_e{config.embed_size}_h{config.hidden_size}"
            wandb.init(project=config.wandb_project, entity=config.wandb_entity, name=default_run_name, config=vars(config))
    
        # Explicitly log the language being used
        wandb.config.update({"language": config.language})
    
    print(f"\nTraining transliteration model for language: {config.language}")
    print(f"Using dataset from: {config.data_dir}\n")
    
    try:
        # Load data
        train_loader, train_dataset = get_dataloader(
            config.train_file, 
            config.batch_size, 
            shuffle=True
        )
        
        val_loader, val_dataset = get_dataloader(
            config.val_file, 
            config.batch_size, 
            shuffle=False
        )
        
        # Create model
        model = Seq2Seq(
            input_size=train_dataset.source_vocab_size,
            output_size=train_dataset.target_vocab_size,
            embed_size=config.embed_size,
            hidden_size=config.hidden_size,
            num_encoder_layers=config.num_encoder_layers,
            num_decoder_layers=config.num_decoder_layers,
            dropout=config.dropout,
            cell_type=config.cell_type
        )
        
        model = model.to(device)
        
        # Define loss function and optimizer
        criterion = nn.CrossEntropyLoss(ignore_index=0)  # Ignore padding index
        optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
        
        # Create directory for saving models
        create_directory(config.model_dir)
        
        best_accuracy = 0.0
        
        # Training loop
        for epoch in range(config.epochs):
            model.train()
            total_loss = 0
            
            pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{config.epochs}')
            
            for batch_idx, batch in enumerate(pbar):
                # Move tensors to device
                source = batch['source'].to(device)
                target = batch['target'].to(device)
                source_lengths = batch['source_lengths']
                
                # Zero the gradients
                optimizer.zero_grad()
                
                # Forward pass
                output = model(source, target, source_lengths, config.teacher_forcing_ratio)
                
                # Calculate loss
                # Reshape output and target for loss calculation
                output_dim = output.shape[-1]
                output = output[:, 1:].reshape(-1, output_dim)  # Remove SOS token
                target = target[:, 1:].reshape(-1)  # Remove SOS token
                
                # Calculate loss
                loss = criterion(output, target)
                
                # Backward pass and optimize
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Clip gradients
                optimizer.step()
                
                total_loss += loss.item()
                pbar.set_postfix({'loss': loss.item()})
            
            # Calculate average loss
            avg_loss = total_loss / len(train_loader)
            
            # Evaluate on validation set
            model.eval()
            val_loss, val_accuracy = evaluate(model, val_loader, criterion, device, train_dataset)
            
            # Log metrics to W&B
            wandb.log({
                'epoch': epoch + 1,
                'train_loss': avg_loss,
                'val_loss': val_loss,
                'val_accuracy': val_accuracy,
                'language': config.language  # Log language with metrics
            })
            
            print(f'Epoch: {epoch+1}, Train Loss: {avg_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')
            
            # Save model if it's the best so far
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
                if not os.path.exists(os.path.join(config.model_dir)):
                    os.makedirs(os.path.join(config.model_dir))

                # Save locally first with language code in filename
                model_path = os.path.join(config.model_dir, f'best_model_{config.language}_{wandb.run.id}.pt')
                save_checkpoint(
                    model, 
                    optimizer, 
                    epoch, 
                    val_accuracy, 
                    model_path
                )
                
                # Save model state dictionary to W&B
                try:
                    # Save model for W&B
                    wandb_model_path = os.path.join(wandb.run.dir, "best_model.pt")
                    torch.save(model.state_dict(), wandb_model_path)
                    
                    # Use W&B Artifact API instead of direct file save
                    artifact = wandb.Artifact(f"model-{config.language}-{wandb.run.id}", type="model")
                    artifact.add_file(wandb_model_path)
                    wandb.log_artifact(artifact)
                    
                    print(f"Model successfully saved to W&B as artifact")
                except Exception as e:
                    print(f"Failed to save model to W&B: {str(e)}")
                    print(f"Model was saved locally to {model_path}")
            
            # Save checkpoint
            if (epoch + 1) % 5 == 0:
                checkpoint_path = os.path.join(config.model_dir, f'checkpoint_{config.language}_{epoch+1}_{wandb.run.id}.pt')
                save_checkpoint(
                    model, 
                    optimizer, 
                    epoch, 
                    val_accuracy, 
                    checkpoint_path
                )

        # Only finish wandb if we started it in this function
        if not run_already_initialized:
            wandb.finish()
        
        return model, best_accuracy
        
    except Exception as e:
        print(f"Error during training: {str(e)}")
        # Only finish wandb if we started it in this function
        if not run_already_initialized:
            wandb.finish()
        # Re-raise the exception for proper error handling
        raise

def evaluate(model, dataloader, criterion, device, dataset):
    model.eval()
    total_loss = 0
    total_correct = 0
    total_examples = 0
    
    with torch.no_grad():
        for batch in dataloader:
            # Move tensors to device
            source = batch['source'].to(device)
            target = batch['target'].to(device)
            source_lengths = batch['source_lengths']
            target_lengths = batch['target_lengths']
            
            # Forward pass
            output = model(source, target, source_lengths, teacher_forcing_ratio=0)
            
            # Calculate loss
            output_dim = output.shape[-1]
            output_for_loss = output[:, 1:].reshape(-1, output_dim)  # Remove SOS token
            target_for_loss = target[:, 1:].reshape(-1)  # Remove SOS token
            
            loss = criterion(output_for_loss, target_for_loss)
            total_loss += loss.item()
            
            # Generate predictions
            predictions = model.predict(
                source,
                source_lengths,
                dataset.target_vocab_size,
                dataset.target_char_to_idx['< SOS >'],
                dataset.target_char_to_idx['<EOS>']
            )
            
            # Calculate accuracy (exact match)
            for i in range(len(predictions)):
                pred_seq = [idx.item() for idx in predictions[i]]
                pred_str = indices_to_string(pred_seq, dataset.target_idx_to_char, dataset.target_char_to_idx['<EOS>'])
                target_str = batch['target_texts'][i]
                
                if pred_str == target_str:
                    total_correct += 1
            
            total_examples += len(predictions)
    
    avg_loss = total_loss / len(dataloader)
    accuracy = total_correct / total_examples
    
    return avg_loss, accuracy

if __name__ == "__main__":
    config = Config()
    train(config)

Base directory: /kaggle/working
Language: hi
Data directory: /kaggle/input/dataset/dakshina_dataset_v1.0/hi/lexicons
Train file exists: True
Val file exists: True
Test file exists: True

Sample train data:
  अं	an	3
  अंकगणित	ankganit	3
  अंकल	uncle	4
  ... (total lines: 44204)

Sample validation data:
  अंकन	ankan	3
  अंगकोर	angkor	3
  अंगिरा	angira	3
  ... (total lines: 4358)

Sample test data:
  अंक	ank	5
  अंक	anka	1
  अंकित	ankit	3
  ... (total lines: 4502)



Training transliteration model for language: hi
Using dataset from: /kaggle/input/dataset/dakshina_dataset_v1.0/hi/lexicons



Epoch 1/30: 100%|██████████| 691/691 [00:28<00:00, 24.63it/s, loss=1.31]


Epoch: 1, Train Loss: 2.2526, Val Loss: 7.6023, Val Accuracy: 0.1212
Model successfully saved to W&B as artifact


Epoch 2/30: 100%|██████████| 691/691 [00:27<00:00, 25.36it/s, loss=0.715]


...

Epoch 29/30: 100%|██████████| 691/691 [00:27<00:00, 25.46it/s, loss=0.0931]


Epoch: 29, Train Loss: 0.1369, Val Loss: 14.8427, Val Accuracy: 0.3983


Epoch 30/30: 100%|██████████| 691/691 [00:27<00:00, 25.35it/s, loss=0.14]  


Epoch: 30, Train Loss: 0.1382, Val Loss: 14.9455, Val Accuracy: 0.3880


epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train_loss,█▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▅▆▇▇▇█████████▇████████▇▇███▇
val_loss,▁▂▃▃▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇████████
epoch,30
language,hi
train_loss,0.13825
val_accuracy,0.38802
val_loss,14.94555


Evaluate on best model

In [ ]:
import torch
import wandb
import os
import json
import argparse
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import pandas as pd
from collections import Counter
from matplotlib.font_manager import FontProperties
import matplotlib as mpl


# from config import Config
# from dataloader import get_dataloader
# from seq2seq import Seq2Seq
# from utils import indices_to_string, create_directory

def evaluate_model(model, dataloader, dataset, device, config):
    model.eval()
    results = []
    total_correct = 0
    total_examples = 0
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            # Move tensors to device
            source = batch['source'].to(device)
            source_lengths = batch['source_lengths']
            source_texts = batch['source_texts']
            target_texts = batch['target_texts']
            
            # Generate predictions
            predictions = model.predict(
                source,
                source_lengths,
                dataset.target_vocab_size,
                dataset.target_char_to_idx['< SOS >'],
                dataset.target_char_to_idx['<EOS>']
            )
            
            # Process each example in the batch
            for i in range(len(predictions)):
                pred_seq = [idx.item() for idx in predictions[i]]
                pred_str = indices_to_string(pred_seq, dataset.target_idx_to_char, dataset.target_char_to_idx['<EOS>'])
                target_str = batch['target_texts'][i]

                # Check if prediction is correct
                is_correct = pred_str == target_str
                if is_correct:
                    total_correct += 1
                
                # Store result
                results.append({
                    'source': source_texts[i],
                    'target': target_texts[i],
                    'prediction': pred_str,
                    'correct': is_correct
                })
                
            total_examples = len(predictions)
    
    # Calculate accuracy
    accuracy = total_correct / total_examples
    
    return results, accuracy

def get_best_model_from_sweep(config, device, sweep_id=None):
    """Get the best model configuration from a completed sweep"""
    
    # Initialize W&B API
    api = wandb.Api()
    
    # Get best run from the project based on validation accuracy
    try:
        if sweep_id is not None:
            sweep = api.sweep(f"{config.wandb_entity}/{config.wandb_project}/{sweep_id}")
        else:
            print('No sweep ID provided. Please provide a valid sweep ID.')
            raise ValueError("Sweep ID is required to find the best model in a sweep.")
        
        best_val_acc = -1
        best_run = None
        
        for run in sweep.runs:
            val_acc = run.summary.get("val_accuracy")
            if val_acc is not None and val_acc > best_val_acc:
                best_val_acc = val_acc
                best_run = run
        
        if best_run is None:
            raise ValueError("No runs with 'val_accuracy' found in the sweep")

        # Extract hyperparameters from the best run
        best_config = {
            'embed_size': best_run.config.get('embed_size', config.embed_size),
            'hidden_size': best_run.config.get('hidden_size', config.hidden_size),
            'num_encoder_layers': best_run.config.get('num_encoder_layers', config.num_encoder_layers),
            'num_decoder_layers': best_run.config.get('num_decoder_layers', config.num_decoder_layers),
            'cell_type': best_run.config.get('cell_type', config.cell_type),
            'dropout': best_run.config.get('dropout', config.dropout)
        }
        
        # Update config with best hyperparameters
        for key, value in best_config.items():
            setattr(config, key, value)
        
        print("Best model hyperparameters:")
        for key, value in best_config.items():
            print(f"  {key}: {value}")
        
        # Try to download the best model
        try:
            # Try to find artifact by name
            artifact = api.artifact(f"{config.wandb_entity}/{config.wandb_project}/model-{config.language}-{best_run.id}:latest")
            artifact_dir = artifact.download()
            best_model_path = os.path.join(artifact_dir, "best_model.pt")
            
            # If artifact download failed, try to get the file directly
            if not os.path.exists(best_model_path):
                best_model_path = best_run.file("best_model.pt").download()
            
            print(f"Downloaded best model from W&B")
        except Exception as e:
            print(f"Failed to download model from W&B: {str(e)}")
            # Look for local model with matching run ID
            model_dir = config.model_dir
            model_files = [f for f in os.listdir(model_dir) if f.startswith(f'best_model_{config.language}_{best_run.id}')]
            
            if model_files:
                best_model_path = os.path.join(model_dir, model_files[0])
                print(f"Using local model: {best_model_path}")
            else:
                # If no matching model, use the most recent local model
                model_files = [f for f in os.listdir(model_dir) if f.startswith(f'best_model_{config.language}_')]
                if model_files:
                    model_files.sort(key=lambda x: os.path.getmtime(os.path.join(model_dir, x)), reverse=True)
                    best_model_path = os.path.join(model_dir, model_files[0])
                    print(f"Using most recent local model instead: {best_model_path}")
                else:
                    raise FileNotFoundError(f"No model file found for language '{config.language}'. Please train a model first.")
        
        return best_config, best_model_path
    
    except Exception as e:
        print(f"Error finding best model: {str(e)}")
        print("Looking for local model instead...")
        
        # Look for local model
        model_dir = config.model_dir
        if not os.path.exists(model_dir):
            raise FileNotFoundError(f"Model directory '{model_dir}' not found")
            
        model_files = [f for f in os.listdir(model_dir) if f.startswith(f'best_model_{config.language}_')]
        
        if model_files:
            # Use the most recent model file
            model_files.sort(key=lambda x: os.path.getmtime(os.path.join(model_dir, x)), reverse=True)
            best_model_path = os.path.join(model_dir, model_files[0])
            print(f"Using local model: {best_model_path}")
            # Keep the default config
            best_config = {
                'embed_size': config.embed_size,
                'hidden_size': config.hidden_size,
                'num_encoder_layers': config.num_encoder_layers,
                'num_decoder_layers': config.num_decoder_layers,
                'cell_type': config.cell_type,
                'dropout': config.dropout
            }
            return best_config, best_model_path
        else:
            raise FileNotFoundError(f"No model file found for language '{config.language}'. Please train a model first.")

def create_prediction_visualizations(results, config):
    """Create visualizations for the prediction results"""
    
    # Try to use a font that supports Devanagari if available
    # First, attempt to find a suitable font
    font_found = False
    
    # List of potential fonts that might support Devanagari
    potential_fonts = ['Nirmala UI', 'Mangal', 'Arial Unicode MS', 'FreeSerif', 'Noto Sans Devanagari']
    
    for font in potential_fonts:
        try:
            # Check if the font is available
            test_prop = FontProperties(family=font)
            if test_prop.get_family()[0] == font:
                # Set this font as the default
                plt.rcParams['font.family'] = font
                print(f"Using font: {font} for Devanagari support")
                font_found = True
                break
        except:
            continue
    
    if not font_found:
        print("Warning: No font with Devanagari support found. Text may not display correctly.")
        # Use a fallback approach - encode special characters
        def sanitize_text(text):
            # Replace Devanagari characters with their Unicode code point representation
            return ''.join([f"U+{ord(c):04X}" if ord(c) > 255 else c for c in text])
    else:
        # If font is found, we can use original text
        def sanitize_text(text):
            return text
    
    # Create a results DataFrame for easier analysis
    df = pd.DataFrame(results)
    
    # Apply sanitization to text columns
    df['source_display'] = df['source'].apply(sanitize_text)
    df['target_display'] = df['target'].apply(sanitize_text)
    df['prediction_display'] = df['prediction'].apply(sanitize_text)
    
    # Calculate overall statistics
    accuracy = df['correct'].mean()
    total_samples = len(df)
    correct_samples_count = df['correct'].sum()
    
    # Handle possible empty groups
    correct_df = df[df['correct'] == True]
    incorrect_df = df[df['correct'] == False]
    
    # Sample safely - ensuring we don't try to sample more items than available
    correct_sample_size = min(5, len(correct_df))
    incorrect_sample_size = min(5, len(incorrect_df))
    
    # Sample from each group
    correct_samples = correct_df.sample(correct_sample_size) if correct_sample_size > 0 else pd.DataFrame()
    incorrect_samples = incorrect_df.sample(incorrect_sample_size) if incorrect_sample_size > 0 else pd.DataFrame()
    
    # Combine samples
    samples = pd.concat([correct_samples, incorrect_samples])
    
    # If we have samples to display, shuffle them
    if len(samples) > 0:
        samples = samples.sample(frac=1)  # Shuffle the combined samples
    
    # Determine the number of plots to display
    n_samples = len(samples)
    
    if n_samples == 0:
        # Handle the case where there are no samples
        fig, ax = plt.subplots(1, 1, figsize=(8, 4))
        ax.text(0.5, 0.5, "No samples available for visualization", 
                horizontalalignment='center', verticalalignment='center')
        ax.axis('off')
    else:
        # Create a grid for the samples
        rows = min(5, max(1, (n_samples + 1) // 2))  # At least 1 row, at most 5 rows
        cols = min(2, n_samples)  # At most 2 columns
        
        fig, axes = plt.subplots(rows, cols, figsize=(16, 10))
        
        # Handle the case of a single subplot
        if rows == 1 and cols == 1:
            axes = np.array([axes])
        
        # Flatten axes array for easier indexing
        axes = np.array(axes).flatten()
        
        # Plot each sample
        for i, (idx, sample) in enumerate(samples.iterrows()):
            if i >= len(axes):
                break
                
            source = sample['source_display']  # Use sanitized text
            target = sample['target_display']  # Use sanitized text
            prediction = sample['prediction_display']  # Use sanitized text
            correct = sample['correct']
            
            bg_color = "#d0f0d0" if correct else "#f0d0d0"
            title_color = "green" if correct else "red"
            
            # Create a table visualization but avoid displaying directly
            # Instead, create a text display
            ax = axes[i]
            ax.axis('off')
            
            # Create text display instead of table
            text_content = f"Source: {source}\nTarget: {target}\nPrediction: {prediction}"
            
            # Display text with background color for prediction
            ax.text(0.5, 0.5, text_content, 
                    horizontalalignment='center', 
                    verticalalignment='center',
                    transform=ax.transAxes,
                    bbox=dict(boxstyle="round,pad=0.5", 
                             facecolor=bg_color if correct else "#f0d0d0",
                             alpha=0.5))
            
            ax.set_title(f"Sample {i+1}: {'Correct' if correct else 'Incorrect'}", color=title_color)
        
        # Hide unused subplots
        for j in range(i+1, len(axes)):
            axes[j].axis('off')
            axes[j].set_visible(False)
    
    plt.tight_layout()
    
    # Save the figure with a more basic approach
    visualization_path = os.path.join(config.prediction_dir, 'prediction_grid.png')
    plt.savefig(visualization_path, bbox_inches='tight')
    plt.close()
    
    # Create error analysis plots
    
    # 1. Correct vs Incorrect counts
    plt.figure(figsize=(10, 6))
    # Fixed: Use hue parameter instead of direct palette assignment
    sns.countplot(x='correct', hue='correct', data=df, palette=["#f0d0d0", "#d0f0d0"], legend=False)
    plt.title(f"Prediction Results: {correct_samples_count} Correct / {total_samples} Total ({accuracy:.2%})")
    plt.xlabel("Prediction Correctness")
    plt.ylabel("Count")
    plt.xticks([0, 1], ["Incorrect", "Correct"])
    
    # Save the figure
    plt.tight_layout()
    error_count_path = os.path.join(config.prediction_dir, 'correct_vs_incorrect.png')
    plt.savefig(error_count_path)
    plt.close()
    
    # 2. Error analysis by string length
    plt.figure(figsize=(12, 6))
    
    # Add string length to DataFrame
    df['source_length'] = df['source'].apply(len)
    df['accuracy_by_length'] = df.groupby('source_length')['correct'].transform('mean')
    
    # Group by length
    length_stats = df.groupby('source_length').agg(
        count=('correct', 'count'),
        accuracy=('correct', 'mean')
    ).reset_index()
    
    # Create the plot - only include lengths with enough samples
    # Get lengths with at least 3 samples or 1% of the dataset
    min_count = max(3, len(df) * 0.01)
    length_stats_filtered = length_stats[length_stats['count'] >= min_count]
    
    if len(length_stats_filtered) > 0:
        # Plot accuracy by length
        plt.bar(length_stats_filtered['source_length'], length_stats_filtered['accuracy'], 
                color='skyblue', edgecolor='black', alpha=0.7)
        plt.xlabel('Source Word Length')
        plt.ylabel('Accuracy')
        plt.title('Accuracy by Word Length')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        
        # Add count labels
        for idx, row in length_stats_filtered.iterrows():
            plt.text(row['source_length'], row['accuracy'] + 0.02, 
                    f"n={row['count']}", ha='center', fontsize=8)
        
        # Improve appearance
        plt.ylim(0, 1.1)
    else:
        # Handle case with no data
        plt.text(0.5, 0.5, "Insufficient data for length analysis", 
                horizontalalignment='center', verticalalignment='center',
                transform=plt.gca().transAxes)
    
    plt.tight_layout()
    
    # Save the figure
    length_analysis_path = os.path.join(config.prediction_dir, 'accuracy_by_length.png')
    plt.savefig(length_analysis_path)
    plt.close()
    
    # 3. Character-level error analysis
    incorrect_df = df[df['correct'] == False].copy()  # Fixed: Create a copy to avoid SettingWithCopyWarning
    
    char_error_path = None
    # Check if there are any incorrect predictions for analysis
    if len(incorrect_df) > 0:
        # Function to find character-level differences - use sanitized text
        def get_char_differences(row):
            target = row['target']
            pred = row['prediction']
            
            # Simple character error analysis
            min_len = min(len(target), len(pred))
            errors = []
            
            for i in range(min_len):
                if target[i] != pred[i]:
                    errors.append((i, target[i], pred[i]))
            
            # Handle length differences
            if len(target) > len(pred):
                for i in range(min_len, len(target)):
                    errors.append((i, target[i], ""))
            elif len(pred) > len(target):
                for i in range(min_len, len(pred)):
                    errors.append((i, "", pred[i]))
                    
            return errors
        
        # Apply function to get character differences
        incorrect_df['char_errors'] = incorrect_df.apply(get_char_differences, axis=1)
        
        # Extract all character errors
        all_errors = []
        for errors in incorrect_df['char_errors']:
            for pos, target_char, pred_char in errors:
                # Use sanitized characters for display
                t_char = sanitize_text(target_char)
                p_char = sanitize_text(pred_char)
                all_errors.append((t_char, p_char))
        
        # Count pair occurrences
        error_counts = Counter(all_errors)
        
        # Get the top 15 most common errors
        top_errors = error_counts.most_common(15)
        
        if top_errors:
            # Create a DataFrame for visualization
            error_df = pd.DataFrame(top_errors, columns=['char_pair', 'count'])
            error_df['target_char'] = error_df['char_pair'].apply(lambda x: x[0] if x[0] else '(empty)')  # Fixed: Use "(empty)" instead of "∅"
            error_df['pred_char'] = error_df['char_pair'].apply(lambda x: x[1] if x[1] else '(empty)')  # Fixed: Use "(empty)" instead of "∅"
            error_df['pair_label'] = error_df.apply(lambda row: f"{row['target_char']} -> {row['pred_char']}", axis=1)  # Fixed: Use "->" instead of "→"
            
            plt.figure(figsize=(14, 8))
            
            # Use a horizontal bar chart for better readability
            bars = plt.barh(error_df['pair_label'], error_df['count'], color='salmon', edgecolor='darkred')
            plt.xlabel('Count')
            plt.ylabel('Character Error (Target -> Prediction)')  # Fixed: Use "->" instead of "→"
            plt.title('Most Common Character-Level Errors')
            plt.grid(axis='x', linestyle='--', alpha=0.7)
            
            # Add count labels
            for bar in bars:
                width = bar.get_width()
                plt.text(width + 0.1, bar.get_y() + bar.get_height()/2, 
                        f'{width}', va='center', fontsize=9)
            
            plt.tight_layout()
            
            # Save the figure
            char_error_path = os.path.join(config.prediction_dir, 'character_errors.png')
            plt.savefig(char_error_path)
            plt.close()
    
    return {
        'prediction_grid': visualization_path,
        'error_count': error_count_path,
        'length_analysis': length_analysis_path,
        'char_error': char_error_path
    }

def main():
    parser = argparse.ArgumentParser(description='Run evaluation for best model in a sweep')
    parser.add_argument('--sweep_id', type=str, help='The sweep ID to use (if already created)')

    config = Config()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    args = parser.parse_args()
    sweep_id = 'v0f3rkuy'#args.sweep_id

    # Create vanilla prediction directory specifically for Question 4
    vanilla_pred_dir = os.path.join(os.path.dirname(config.prediction_dir), "predictions_vanilla")
    if not os.path.exists(vanilla_pred_dir):
        os.makedirs(vanilla_pred_dir, exist_ok=True)
    
    # Temporarily override the prediction directory
    original_pred_dir = config.prediction_dir
    config.prediction_dir = vanilla_pred_dir
    
    # Initialize W&B for evaluation
    wandb.init(project=config.wandb_project, entity=config.wandb_entity, job_type="evaluation", 
               name=f"vanilla_eval_{config.language}")
    
    # Log the language being evaluated
    wandb.config.update({"language": config.language})
    
    # Load test data
    test_loader, test_dataset = get_dataloader(
        config.test_file, 
        config.batch_size, 
        shuffle=False
    )
    
    # Get the best model from sweep
    best_config, best_model_path = get_best_model_from_sweep(config, device, sweep_id)

    # Load the model checkpoint to get the correct vocabulary size
    checkpoint = torch.load(best_model_path, map_location=device)

    # Extract the vocabulary size from the model checkpoint
    output_size = checkpoint['decoder.fc_out.bias'].size(0)  # This should be 66 based on the error

    # Create model with the best configuration and correct vocabulary size
    model = Seq2Seq(
        input_size=test_dataset.source_vocab_size,
        output_size=output_size,  # Use the output size from the checkpoint instead
        embed_size=best_config['embed_size'],
        hidden_size=best_config['hidden_size'],
        num_encoder_layers=best_config['num_encoder_layers'],
        num_decoder_layers=best_config['num_decoder_layers'],
        dropout=best_config['dropout'],
        cell_type=best_config['cell_type']
    )
    
    # Load the model weights
    model.load_state_dict(torch.load(best_model_path, map_location=device))
    model = model.to(device)
    
    # Create prediction directory
    create_directory(config.prediction_dir)
    
    # Evaluate model
    results, accuracy = evaluate_model(model, test_loader, test_dataset, device, config)
    
    # Create visualizations
    visualization_paths = create_prediction_visualizations(results, config)
    
    # Save results
    output_file = os.path.join(config.prediction_dir, f"predictions_{config.language}_{wandb.run.id}.json")
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    
    # Log accuracy to W&B
    wandb.log({'test_accuracy': accuracy, 'language': config.language})
    
    print(f'Test Accuracy for {config.language}: {accuracy:.4f}')
    print(f'Predictions saved to {output_file}')
    
    # Create a table for W&B
    columns = ["source", "target", "prediction", "correct"]
    data = [[r["source"], r["target"], r["prediction"], r["correct"]] for r in results]
    table = wandb.Table(columns=columns, data=data)
    wandb.log({"predictions": table})
    
    # Log the visualization as W&B images
    for name, path in visualization_paths.items():
        if path and os.path.exists(path):
            image = wandb.Image(path)
            wandb.log({name: image})
            print(f"logged {name} to W&B from {path}")
    
    # Log some sample predictions
    samples = results[:10]  # First 10 samples
    sample_data = "\n".join([f"Source: {s['source']}, Target: {s['target']}, Prediction: {s['prediction']}, Correct: {s['correct']}" for s in samples])
    wandb.log({"samples": wandb.Html(sample_data)})
    
    # Analyze error patterns
    error_analysis = analyze_errors(results)
    wandb.log({"error_analysis": wandb.Html(error_analysis)})
    
    # Restore original prediction directory
    config.prediction_dir = original_pred_dir
    
    wandb.finish()

def analyze_errors(results):
    """Analyze common error patterns in the predictions"""
    
    df = pd.DataFrame(results)
    
    # Filter for incorrect predictions
    incorrect = df[df['correct'] == False].copy()  # Fixed: Create a copy to avoid SettingWithCopyWarning
    
    if len(incorrect) == 0:
        return "<p>No errors to analyze!</p>"
    
    # Error patterns analysis
    analysis = "<h3>Error Analysis</h3>"
    
    # 1. Length differences
    incorrect.loc[:, 'target_len'] = incorrect['target'].apply(len)  # Fixed: Use .loc to avoid SettingWithCopyWarning
    incorrect.loc[:, 'pred_len'] = incorrect['prediction'].apply(len)  # Fixed: Use .loc to avoid SettingWithCopyWarning
    incorrect.loc[:, 'len_diff'] = incorrect['pred_len'] - incorrect['target_len']  # Fixed: Use .loc to avoid SettingWithCopyWarning
    
    len_diff_counts = incorrect['len_diff'].value_counts().head(5)
    len_diff_html = "<h4>Length Differences in Incorrect Predictions</h4>"
    len_diff_html += "<ul>"
    
    for diff, count in len_diff_counts.items():
        len_diff_html += f"<li>Difference of {diff} characters: {count} occurrences ({(count/len(incorrect))*100:.1f}%)</li>"
    
    len_diff_html += "</ul>"
    analysis += len_diff_html
    
    # 2. Example error cases for inspection
    analysis += "<h4>Sample Error Cases</h4>"
    analysis += "<table border='1' style='border-collapse: collapse;'>"
    analysis += "<tr><th>Source</th><th>Target</th><th>Prediction</th><th>Analysis</th></tr>"
    
    # Select diverse error examples
    error_samples = []
    
    # Shorter prediction
    shorter_pred = incorrect[incorrect['len_diff'] < 0].head(1)
    if not shorter_pred.empty:
        error_samples.append(shorter_pred.iloc[0])
    
    # Longer prediction
    longer_pred = incorrect[incorrect['len_diff'] > 0].head(1)
    if not longer_pred.empty:
        error_samples.append(longer_pred.iloc[0])
    
    # Same length but incorrect
    same_len = incorrect[incorrect['len_diff'] == 0].head(1)
    if not same_len.empty:
        error_samples.append(same_len.iloc[0])
    
    # Add more examples if needed
    if len(error_samples) < 3:
        more_examples = incorrect.head(3 - len(error_samples))
        error_samples.extend(more_examples.to_dict('records'))
    
    for sample in error_samples:
        source = sample['source']
        target = sample['target']
        prediction = sample['prediction']
        
        # Simple error analysis
        analysis_text = ""
        
        if len(prediction) < len(target):
            analysis_text = f"Missing characters (prediction too short by {len(target) - len(prediction)} chars)"
        elif len(prediction) > len(target):
            analysis_text = f"Extra characters (prediction too long by {len(prediction) - len(target)} chars)"
        else:
            # Find positions of difference
            diff_positions = []
            for i, (t, p) in enumerate(zip(target, prediction)):
                if t != p:
                    diff_positions.append(i)
            analysis_text = f"Character mismatch at positions: {diff_positions}"
        
        analysis += f"<tr><td>{source}</td><td>{target}</td><td>{prediction}</td><td>{analysis_text}</td></tr>"
    
    analysis += "</table>"
    
    return analysis

if __name__ == "__main__":
    main()